In [1]:
from TrajGen import (
    TrajectoryGenerator,
    DataPoint,
    TrajectoryGeneratorCollection,
    WayPoint,
    color_hex,
    rand_24_bit,
    plot_trajectory,
    plot_datapoint,
    setup_map,
)
import os
import gpxpy
import folium
import random
from typing import Tuple, List
from folium import plugins
import matplotlib.pyplot as plt
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import uuid
import utm 
import gpxpy
import json
from shapely.geometry import Point, Polygon, LineString
from shapely import ops

center = [55.39594, 10.38831]

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [2]:
# kmh_to_ms = (1000 / (60 * 60))
# # Set trajectory charateristics
# mean_speed = 4.0 * kmh_to_ms  # Average human walking speed in m/s
# std_speed = 0.5 * kmh_to_ms   # Deviation in walking speed in m/s
# mean_time_delta = 5.
# std_time_delta = 20.

# TESTTING !!!!!
kmh_to_ms = (1000 / (60 * 60))
# Set trajectory charateristics
mean_speed = 1. * kmh_to_ms  # Average human walking speed in m/s
std_speed = 0.0 * kmh_to_ms   # Deviation in walking speed in m/s
mean_time_delta = 5.
std_time_delta = 20.


def ncrands(n: int, min_: int, max_: int):
    rs = []; r0 = np.random.randint(min_, max_)
    for _ in range(n):
        r1 = np.random.randint(min_, max_ - 1) 
        r2 = (r0 + r1 + 1) % max_
        r0 = r2
        rs.append(r2)
    return rs

# def haversine_distance(
#         lat_1: float, lon_1: float,
#         lat_2: float, lon_2: float,
#     ) -> float:
#     EARTH_RADIUS = 6371. * 1000.
#     """Calculate the 'Haversine' (great-circle) distance in meters between two locations
#     / (latitude, longitude) points.

#     Args:
#         lat_1 (float): Latitude of location 1.
#         lon_1 (float): Longitude of location 1.
#         lat_2 (float): Latitude of location 2.
#         lon_2 (float): Longitude of location 2.

#     Returns:
#         float: The distance between location 1 and 2 in meters. 
#     """
#     d_lat = np.radians(lat_1 - lat_2)
#     d_lon = np.radians(lon_1 - lon_2)
#     lat1 = np.radians(lat_1)
#     lat2 = np.radians(lat_2)
#     a = np.sin(d_lat / 2) * np.sin(d_lat / 2) + \
#         np.sin(d_lon / 2) * np.sin(d_lon / 2) * np.cos(lat1) * np.cos(lat2)
#     c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
#     d = EARTH_RADIUS * c
#     return d

In [3]:
# Used for generating: Normal trajectories and trajectories with pacing behavior
start_latitude = 55.375106225495145
start_longitude = 10.396267369060276

start_middle_latitude = 55.38169472786001
start_middle_longitude = 10.395885727870679

end_middle_latitude = 55.387728473654015
end_middle_longitude = 10.396407680375694

end_latitude = 55.39472434817015
end_longitude = 10.393096102223307

# Used for generating: Normal trajectories and deviating trajectories
middle_latitude = np.abs((start_middle_latitude - end_middle_latitude)) / 2 + start_middle_latitude
middle_longitude = np.abs((start_middle_longitude - end_middle_longitude)) / 2 + start_middle_longitude

In [4]:
# def to_dataframe(relative_path):
#     EPOCH = datetime.fromtimestamp(0)
#     sorted_data = list()
#     for gpx_file in os.listdir(relative_path):
#         filepath = os.path.join(relative_path, gpx_file)
#         with open(filepath, "r") as f:
#             gpx = gpxpy.parse(f)
#             for track in gpx.tracks:
#                 for segment in track.segments:
#                     for point in segment.points:
#                         # Make timestamp a naive datetime object
#                         point.time = point.time.replace(tzinfo=None)
#                         d = {
#                             "timestamp": (point.time - EPOCH).total_seconds(),
#                             "point": point,
#                         }
#                         sorted_data.append(d)
#     sorted_data.sort(key = lambda x: x.get("timestamp"))
#     points = []
#     for item in sorted_data:
#         data = item["point"]
#         p = {
#             "lat": data.latitude,
#             "lon": data.longitude,
#             "time": data.time,
#         }
#         points.append(p)
#     return pd.DataFrame(data = points)

# # Read in location data
# rel_path = "./gpsdata"
# df = to_dataframe("./gpsdata")

In [5]:
# # Estimate parameters
# dts = [0]
# dxs = [0]

# for i in range(25000, df.shape[0]):
#     row0 = df.iloc[i - 1]
#     row1 = df.iloc[i]
#     dt = (row1["time"].to_pydatetime() - row0["time"].to_pydatetime()).total_seconds()    
#     dx = haversine_distance(
#         lat_1 = row0["lat"], lon_1 = row0["lon"],
#         lat_2 = row1["lat"], lon_2 = row1["lon"],
#     )
#     if (dt > 100. or dx > 500.) or dx < 5.0:
#         pass
#     else:
#         dxs.append(dx)
#         dts.append(dt)

In [6]:
# np.mean(dts)
# speeds = []
# for i in range(len(dts)):
#     if dts[i] == 0:
#         pass
#         speeds.append(0)
#     else:
#         speeds.append(dxs[i] / dts[i])
# np.mean(speeds)
# df
# plt.hist(dts, density=True, bins=10)  # density=False would make counts
# plt.ylabel('Probability')
# plt.xlabel('Data');

In [7]:
# def generate_looping_patterns(seed: int, n_normal = 5, n_looping = 5, repeat_loop = 1):
#     np.random.seed(seed)
#     normal_pattern = [
#         WayPoint(
#             latitude = start_latitude,
#             longitude = start_longitude,
#             duration = 240, # In seconds
#             std = 14, # In meters
#         ),
#         WayPoint(
#             latitude = end_latitude,
#             longitude = end_longitude,
#             duration = 240, # In seconds
#             std = 14, # In meters
#         ),
#         WayPoint(
#             latitude = start_latitude,
#             longitude = start_longitude,
#             duration = 240, # In seconds
#             std = 14, # In meters
#         ),
#     ]
#     list_waypoints = []
#     # 5 trajectories that visit the following waypoints in order:
#     # --> start waypoint --> end waypoint --> start waypoint
#     list_waypoints.append((False, n_normal, normal_pattern))
#     # A looping pattern go between the following waypoints in a random order:
#     options = [
#             WayPoint(
#                 latitude = start_middle_latitude,
#                 longitude = start_middle_longitude,
#                 duration = 0, # In seconds
#                 std = 0, # In meters
#             ),
#             WayPoint(
#                 latitude = end_middle_latitude,
#                 longitude = end_middle_longitude,
#                 duration = 0, # In seconds
#                 std = 0, # In meters
#             ),    
#             WayPoint(
#                 latitude = end_latitude,
#                 longitude = end_longitude,
#                 duration = 0, # In seconds
#                 std = 0, # In meters
#             ),    
#     ]
#     # Generate several looping patterns:
#     looping_pattern = [
#         WayPoint(
#             latitude = start_latitude,
#             longitude = start_longitude,
#             duration = 240, # In seconds
#             std = 14, # In meters
#         ),
#     ]
#     rands = ncrands(
#         2 * repeat_loop,
#         min_ = 0,
#         max_ = len(options),
#     )
#     for i in range(2 * repeat_loop):
#         looping_pattern.extend([options[i]])
#     looping_pattern.append(
#         WayPoint(
#             latitude = start_latitude,
#             longitude = start_longitude,
#             duration = 240, # In seconds
#             std = 14, # In meters
#         )
#     )
#     list_waypoints.append((True, 1, looping_pattern))
#     return list_waypoints 

# looping_pattern_waypoints_list = generate_looping_patterns(
#     seed = 12,
#     n_normal = 5,
#     n_looping = 5, 
# )

# # Generate and plot generated data
# map_ = setup_map(center)
# map_ = plot_datapoint([start_latitude, start_longitude], color = "red", map_ = map_)
# map_ = plot_datapoint([start_middle_latitude, start_middle_longitude], color = "green", map_ = map_)
# map_ = plot_datapoint([end_middle_latitude, end_middle_longitude], color = "blue", map_ = map_)
# map_ = plot_datapoint([end_latitude, end_longitude], color = "cyan", map_ = map_)

# trajectories0 = []
# for anomaly, n, waypoints in looping_pattern_waypoints_list:
#     for _ in range(n):
#         random.seed(time.time())
#         color = "#" + str(color_hex(num = rand_24_bit()))
#         trajectory_generator = TrajectoryGenerator(
#             waypoints = waypoints,
#             mean_time_delta = mean_time_delta,
#             std_time_delta = std_time_delta,
#             mean_speed = mean_speed,
#             std_speed = std_speed,
#             std_datapoint = 14,
#         )
#         map_ = plot_trajectory(
#             trajectory_generator.to_latlon(),
#             color = color,
#             map_ = map_,
#         )
#         trajectories0.append((anomaly, trajectory_generator))
# map_

In [8]:
def generate_deviating_patterns(seed: int, n_normal = 10, n_deviating = 5):
    np.random.seed(seed)
    init_pair = [
        WayPoint(
            latitude = start_latitude,
            longitude = start_longitude,
            duration = 60, # In seconds
            std = 0, # In meters
#             duration = 240, # In seconds
#             std = 14, # In meters
        ),
    ]
    list_waypoints = []
    x, y, zone, lat_band  = utm.from_latlon(middle_latitude, middle_longitude)
    for _ in range(n_deviating):
        while True:
            dx0 = np.random.normal(0, 600)
            dx1 = np.random.normal(0, 600)
            if np.abs(dx0) > 200 and dx1 > 0:
                break
        lat, lon = utm.to_latlon(dx0 + x, dx1 + y, zone, lat_band)
        list_waypoints.append(
            (True, 1, 
                init_pair + \
                 [WayPoint(
                    latitude = lat,
                    longitude = lon,
                    duration = 0, # Seconds
                    std = 0, # In meters
                )] + \
                 [WayPoint(
                    latitude = end_latitude,
                    longitude = end_longitude,
                    duration = 60, # In seconds
                    std = 0, # In meters
#                     duration = 240, # In seconds
#                     std = 14, # In meters
                )],
            )
        )
    list_waypoints.append(
        (False, n_normal, 
            init_pair + [WayPoint(
                latitude = end_latitude,
                longitude = end_longitude,
                duration = 60, # In seconds
                std = 0, # In meters
            )],
        )
    )
    return list_waypoints
   
deviating_patterns_waypoint_list = generate_deviating_patterns(
    seed = 2,
    n_normal = 10,
    n_deviating = 5, 
)

map_ = setup_map(center)
map_ = plot_datapoint([start_latitude, start_longitude], color = "red", map_ = map_)
map_ = plot_datapoint([middle_latitude, middle_longitude], color = "green", map_ = map_)
map_ = plot_datapoint([end_latitude, end_longitude], color = "green", map_ = map_)

trajectories1 = []
for anomaly, n, waypoints in deviating_patterns_waypoint_list:
    for _ in range(n):
        random.seed(time.time())
        color = "#" + str(color_hex(num = rand_24_bit()))
        print(waypoints)
        trajectory_generator = TrajectoryGenerator(
            waypoints = waypoints,
            mean_time_delta = mean_time_delta,
            std_time_delta = std_time_delta,
            mean_speed = mean_speed,
            std_speed = std_speed,
            # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0219890
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6638960/
            # np.mean(np.abs(np.random.normal(0, 9, size=1000))) approx 7
            std_datapoint = 9,
        )
        map_ = plot_trajectory(
            trajectory_generator.to_latlon(),
            color = color,
            map_ = map_,
        )
        trajectories1.append((anomaly, trajectory_generator))
map_

[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e3070>, <TrajGen.WayPoint object at 0x7f0af01e3160>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e3fa0>, <TrajGen.WayPoint object at 0x7f0af01e8cd0>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e8d30>, <TrajGen.WayPoint object at 0x7f0af01e8d90>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e8df0>, <TrajGen.WayPoint object at 0x7f0af01e8e50>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e8eb0>, <TrajGen.WayPoint object at 0x7f0af01e8f10>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e3130>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e3130>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <TrajGen.WayPoint object at 0x7f0af01e3130>]
[<TrajGen.WayPoint object at 0x7f0b010a9790>, <

In [9]:
def generate_repeated_pattern(repeat: int = 4):
    start_waypoint = [55.37422930819445, 10.395805527421492]
    middle_waypoint = [55.38153960807826, 10.395926910342931]
    end_waypoint = [55.38217747110452, 10.410523206645504]
    start_pair = WayPoint(
        latitude = start_waypoint[0],
        longitude = start_waypoint[1],
        duration = 1200, # In seconds
        std = 8.75, # In meters
    )
    middle_pair = WayPoint(
        latitude = middle_waypoint[0],
        longitude = middle_waypoint[1],
        duration = 0, # In seconds
        std = 8.75, # In meters
    )
    end_pair = WayPoint(
        latitude = end_waypoint[0],
        longitude = end_waypoint[1],
        duration = 1200, # In seconds
        std = 8.75, # In meters
    )
    trip = []; list_waypoints = []
    profile = [start_pair, middle_pair, end_pair] 
    # Start
    trip.append(profile[0])
    # Following...
    for i in range(repeat):
        trip.append(profile[1])
        trip.append(profile[2])
        trip.append(profile[1])
        trip.append(profile[0])
    list_waypoints.append((False, 1, trip))    
    return list_waypoints
   
repeated_patterns_waypoint_list = generate_repeated_pattern(
    repeat = 5,
)
map_ = setup_map(center)
# # map_ = plot_datapoint([start_latitude, start_longitude], color = "red", map_ = map_)
# # map_ = plot_datapoint([middle_latitude, middle_longitude], color = "green", map_ = map_)
# # map_ = plot_datapoint([end_latitude, end_longitude], color = "green", map_ = map_)

trajectories2 = []
for anomaly, n, waypoints in repeated_patterns_waypoint_list:
    for _ in range(n):
        random.seed(time.time())
        color = "#" + str(color_hex(num = rand_24_bit()))
        print(waypoints)
        trajectory_generator = TrajectoryGenerator(
            waypoints = waypoints,
            mean_time_delta = mean_time_delta,
            std_time_delta = std_time_delta,
            mean_speed = mean_speed,
            std_speed = std_speed,
            # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0219890
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6638960/
            # np.mean(np.abs(np.random.normal(0, 9, size=1000))) approx 7
            std_datapoint = 8.75,
        )
        map_ = plot_trajectory(
            trajectory_generator.to_latlon(),
            color = color,
            map_ = map_,
        )
        trajectories2.append((anomaly, trajectory_generator))
map_

[<TrajGen.WayPoint object at 0x7f0aebf4ceb0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4cfa0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4ceb0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4cfa0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4ceb0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4cfa0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4ceb0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4cfa0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4ceb0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4cfa0>, <TrajGen.WayPoint object at 0x7f0aebf4cee0>, <TrajGen.WayPoint object at 0x7f0aebf4ceb0>]


In [10]:
tgc = TrajectoryGeneratorCollection(
    trajectory_generators = trajectories2,
    start_datetime = datetime.now(),
    gap = timedelta(days = 1.),
)
main_df = tgc.to_dataframe(anom_start = False)
batches0 = [main_df.to_json(orient = "split")]
with open("preprocessing_case_test.json", "w") as f:
    f.write(json.dumps(batches0))

In [11]:
def generate_trajectories(
    waypoints: List,
    start_datetime: datetime,
    gap: timedelta = timedelta(days = 1.),
    *args, **kwargs,
    ) -> TrajectoryGeneratorCollection: 
    trajectories = []
    for anomaly, n, waypoints in waypoints:
        for _ in range(n):
            random.seed(time.time())
            color = "#" + str(color_hex(num = rand_24_bit()))
            trajectory_generator = TrajectoryGenerator(waypoints = waypoints, *args, **kwargs)
            trajectories.append((anomaly, trajectory_generator))
    return TrajectoryGeneratorCollection(
        trajectory_generators = trajectories,
        start_datetime = start_datetime,
        gap = gap,
    ) 

In [12]:
def generate_batches_deviating(n_batches, seeds):
    batches = []
    for seed in seeds:
        waypoints = generate_deviating_patterns(
            seed = seed,
            n_normal = 10,
            n_deviating = 5, 
        ) 
        kwargs = {
            "mean_time_delta": mean_time_delta,
            "std_time_delta": std_time_delta,
            "mean_speed": mean_speed,
            "std_speed": std_speed,
            "std_datapoint": 9, # 9
        }
        tgc = generate_trajectories(
            waypoints = waypoints, 
            start_datetime = datetime.now(),
            **kwargs,
        )
        main_df = tgc.to_dataframe(anom_start = True)
        batches.append(main_df.to_json(orient = "split"))
    return batches


# def read_batches(data_in):
# #     data_in = None; #df = 
# #     with open(json_file, "r") as f:
# #         data_in = json.loads(f.read())
#     batches = {}; 
#     for i, batch in enumerate(data_in): # Get each batch of trajectories
#         df_batch = pd.read_json(batch, orient = "split")
#         _data = []
#         for _, row in df_batch.iterrows():
#             df_traj = pd.read_json(row["df"], orient = "split")
#             _data.append(df_traj)
#         df_batch["df"] = _data
#         batches[i] = df_batch 
#     return batches

    
# def generate_batches_looping(n_batches):
#     batches = []
#     seeds = [i for i in range(12, 12 + n_batches + 1)]
#     for seed in seeds:
#         waypoints = generate_looping_patterns(
#             seed = seed,
#             n_normal = 10,
#             n_looping = 5, 
#         ) 
#         kwargs = {
#             "mean_time_delta": mean_time_delta,
#             "std_time_delta": std_time_delta,
#             "mean_speed": mean_speed,
#             "std_speed": std_speed,
#             "std_datapoint": 14,
#         }
#         tgc = generate_trajectories(
#             waypoints = waypoints, 
#             start_datetime = datetime.now(),
#             **kwargs,
#         )
#         main_df = tgc.to_dataframe()
#         batches.append(main_df.to_json(orient = "split"))
#     return batches

# batches0 = generate_batches_deviating(n_batches = 1)

In [13]:
start_seed = 13
seeds = [2, 3, 5, 6, 8] # [i for i in range(start_seed, start_seed + n_batches + 1)]
n_batches = len(seeds)
batches0 = generate_batches_deviating(n_batches = n_batches, seeds = seeds)
with open("main_data_deviating.json", "w") as f:
    f.write(json.dumps(batches0))

In [14]:
# batches1 = generate_batches_looping(n_batches = 5)
# with open("main_data_looping.json", "w") as f:
#     f.write(json.dumps(batches1))

In [15]:
# plt.plot(x, y, c = "blue")
# plt.scatter([anom_point[0]], [anom_point[1]], c = "red")